# Lab 5: Adversarial examples

In this lab we will look at generating adversarial examples using the [Fast Sign Gradient Method (FGSM)](https://arxiv.org/abs/1412.6572) developed by [Ian Goodfellow](https://research.google.com/pubs/105214.html).

By the end of this lab you should have...

1. Generated some adversarial examples
2. Viewed those adversarial examples in tensorboard and compared them with their initial counterparts
3. Utilised adversarial generation in training to improve the robustness of your network against adversarial attacks

## Introduction


![Adversarial Example](./media/adversarial-example.png)

Adversarial examples are examples that have undergone a slight pertubation causing the predicted label to wildly differ from the label predicted on the original unpertubed example yet visually look very similar to the original examples.

Although the classifier you’ve trained in the previous labs works well on the provided test images, it is possible to take an example image $\mathbf{x}$ and add a tensor $\mathbf{\eta}$ of the same shape containing many small changes (known as pertubations) and construct an *adversarial example* $\mathbf{\widetilde{x}} = \mathbf{x} + \mathbf{\eta}$. How do we construct $\mathbf{\eta}$ such that $\mathbf{\widetilde{x}}$ is misclassified?

For instance, in the above example the original panda image is classified correctly with 57.7% accuracy. When adding the filter shown in the center to the original image there is barely a noticable change in the image and the result is still easily recognisable as a panda for humans, yet the new image is now classified by the network as a *gibbon* with 99.3% confidence.

The adversarial image in the above example is generated using the "fast gradient sign method" (fgsm) developed by Goodfellow et al. This method creates small but intentionally worst-case perturbations to images in the original dataset such that the classifier outputs an incorrect answer with high confidence.

Consider the loss function we use to train the neural network: $J(\mathbf{\theta}, \mathbf{x}, y)$; $\mathbf{\theta}$, are the parameters/weights of the network, $\mathbf{x}$, the input image, and $y$, the correct class label of $\mathbf{x}$. In training, we compute the partial derivative of the loss function with respect to the parameters, however there isn't anything stopping us from computing the partial derivative of the loss function with respect to each pixel in the input image: $\frac{\partial J}{\partial x}$ where $x$ is a pixel in the input image $\mathbf{x}$. The sign of $\left.\frac{\partial J}{\partial x}\right\vert_{\mathbf{\theta}, \mathbf{x}}$ (the partial derivative of the loss with respect to each input pixel $x$, evaluated at $\mathbf{x}$ and $\mathbf{\theta}$) tells us whether increasing pixel $x$ increases the loss or not; if the sign of this is positive then increasing $x$ increases the loss, if it is negative then increasing $x$ decreases the loss. We denote the tensor of all partial derivatives of $J$ w.r.t each pixel $x$ in $\mathbf{x}$ as $\nabla_{\mathbf{x}}J(\mathbf{\theta}, \mathbf{x}, y)$.
$\DeclareMathOperator{\sign}{sign}$

Armed with this information we can determine a setting of $\mathbf{\eta}$ that produces our adversarial example. We want our adversarial image to **increase the loss (i.e. fool the network)** so lets define $\mathbf{\widetilde{x}}$ to take a step $\epsilon$ in the direction of the increasing gradient from $\mathbf{x}$ yielding 

$$\eta = \epsilon \sign(\nabla_{\mathbf{x}}J(\mathbf{\theta}, \mathbf{x}, y))$$

$$\mathbf{\widetilde{x}} = \mathbf{x} + \mathbf{\eta}$$

We have to pick a value for $\epsilon$ such that the new image $\mathbf{\widetilde{x}}$ is visually similar to the the original $\mathbf{x}$, but that fools the network by increasing the loss. These two factors act against each other, we want a small $\epsilon$ so the adversarial examples does not look different, but it also needs to be large enough to cause a significant dip in classification accuracy. You'll have to play around with values of $\epsilon$ to find out what works best. Try starting off with $\epsilon = 0.1$.

## Set up

We're going to make use of a library of NN attacks for tensorflow called [CleverHans](https://github.com/tensorflow/cleverhans) ([docs here](https://cleverhans.readthedocs.io/en/v.2.1.0/source/attacks.html)). CleverHans contains a variety of attacks such as the fast gradient sign method (incidently the fgsm code in this library was written by Ian Goodfellow!)

Install cleverhans using pip:

```
$ module add libs/tensorflow/1.2
$ pip2 install --user -e git+http://github.com/tensorflow/cleverhans.git@80e57#egg=cleverhans
```

You should now be able to import `cleverhans` in python, check with:

```
$ python -c 'import cleverhans; print(cleverhans.__version__)' 
2.0.0-282675e7a58ad510a0ef523f64c946dd
```

## Generating adversarial examples

CleverHans has a few tutorials showing how to use the library, have a look at the [MNIST adversarial tutorial](https://github.com/tensorflow/cleverhans/blob/v2.0.0/cleverhans_tutorials/mnist_tutorial_tf.py) to get an idea of how to use the library.

**NOW** edit your code as follows.

1. Rename your code from Lab 4 into `cifar_adversarial.py`
2. Add the following imports to your code
```python
from cleverhans.attacks import FastGradientMethod
from cleverhans.model import CallableModelWrapper
```
3. Add `cifar.preprocess()` after initialising the `cifar` variable as follows. This will preprocess images to be within the range [0, 1].
   ```python            
        cifar = cf.cifar10(batchSize=FLAGS.batch_size)
        cifar.preprocess()  # necessary for adversarial attack to work well.
   ```    
4. Wrap your `deepnn` call with a variable_scope block so we can reuse the graph later, then wrap your model with `CallableModelWrapper` for use with cleverhans in the same variable_scope block. The second argument tells cleverhans whether your model returns probabilities, `'probs'` (post-softmax layer), or logits, `'logits'`, (pre-softmax layer).
   ```python            
   with tf.variable_scope('model'):
       y_conv = deepnn(x_image)
       model = CallableModelWrapper(deepnn, 'logits')
   ```           
5. In your `main` function, within your session, construct a `FastGradientMethod` object, we reuse the `'model'` variable scope so that the fast gradient sign control flow graph is constructed around the existing `deepnn` graph rather than creating a copy of it:
   ```python
   with tf.variable_scope('model', reuse=True):
       fgsm = FastGradientMethod(model, sess=sess)
   ```
6. You can use this object to construct a batch of adversarial examples `x_adv` from a batch of examples `x` (which must be a 4D tensor):
   ```python
       x_adv = fgsm.generate(x, eps=0.05, clip_min=0.0, clip_max=1.0)
   ```
The `clip_min` and `clip_max` keywords are used to limit the minimum and maximum values the generated values can have, we want to bound them to the interval $[0, 1]$ as the input training data is preprocessed to be within these bounds.

`x_adv` is a tensor, so you will have to evaluate it to get a concrete `np.ndarray` out of it. 


---
![](./media/error.png)

Gotchas to watch out for
- If you're using `tf.layers.batch_normalization` layers, you'll need to pass the `name='XXXX'` kwarg to set the name of the parameters defined in the layer, otherwise you'll get an error like 
```
ValueError: Variable model/batch_normalization_1/beta does not exist, or was not created with tf.get_variable(). Did you mean to set reuse=None in VarScope?`
```
- Your `deepnn` function should return only the logits tensor, nothing else, if you still return `img_summary` as well, you'll get an error like:
  ```
  TypeError: Cannot convert a list containing a tensor of dtype <dtype: 'string'> to <dtype: 'float32'> (Tensor is: <tf.Tensor 'model_1/Input_images_1:0' shape=() dtype=string>)
  ```

### Practical 5.1: Generate adversarial examples

**NOW** Generate adversarial examples for every image in the *test set* using the `generate` method from the `fgsm` object using the ```eps=0.05``` defined above, and evaluate the accuracy of the classifier on the adversarial examples *only*. Compare the accuracy on the adversarial examples with the accuracy on the original test set, e.g.

```
test_set: 0.751
adversarial_test_set: 0.143
```

## Inspecting adversarial examples

### Practical 5.2: Record image summaries for adversarial examples

You can write out your test images, as well as your adversarial images, and see them in the *Images* tab of tensorflow as follows:

1. Define an image summary (make sure the tensor is a 4D batch of images, and not the 2D batch returned by the `cifar` class) for both the test images and the adversarial images.
   ```python
   test_img_summary = tf.summary.image('Test Images', x_image)
   adv_test_img_summary = tf.summary.image('Adversarial test Images', x_adv)
   ```
2. You can merge these to create a single summary op to run
   ```python
   adv_summary = tf.summary.merge([test_img_summary, adv_test_img_summary])
   ```
3. Define another summary writer using a [`SummaryWriter`](https://www.tensorflow.org/api_docs/python/tf/summary/FileWriter) named `adversarial_writer` as follows
   ```python
   adversarial_writer = tf.summary.FileWriter(run_log_dir + "_adversarial", sess.graph)
   ```
3. Run the `adv_summary` op during testing and log the results to the `adversarial_writer`
   ```python
   adv_summary_str = sess.run([adv_summary], feed_dict={x: testImages, y_: testLabels})
   test_writer.add_summary(adv_summary_str, step)
   ```

Once you've managed to write out the adversarial images to a summary you should see them in the *Images* tab of tensorboard. 

**NOW** Try tweaking the $\epsilon$ value (`eps`) and see how it affects the generated adversarial examples and accuracy on them.

Pick the 3 best adversarial examples (those that are most visually similar to the examples they were generated from) and save the original examples as `x1.png`, `x2.png`, and `x3.png`; and the adversarial examples as `x1_adv.png`, `x2_adv.png`, and `x3_adv.png`. Compute an image diff between `xn.png` and `xn_adv.png` for all `n` using ImageMagick's compare (see below for details). 


#### Computing image diffs using `compare`

ImageMagick contains a utility called `compare` that can be used to compute an image diff, e.g. `compare in1.png in2.jpg in_diff.jpg`.

`compare` is available on BC4, snowy and the lab machines, copy the images you wish to diff over to any of these servers and run `compare`, alternatively install it for your platform:

- Ubuntu: `sudo apt-get update; sudo apt-get install -y imagemagick`
- macOS: `sudo brew install imagemagick` 
- Windows: `choco install -y imagemagick` (or get a binary from here: https://www.imagemagick.org/download/binaries/ImageMagick-7.0.7-11-Q16-x64-dll.exe)

---
![](./media/error.png)

### Gotchas to watch out for

- Help! My images look like noise, not like the test images:

![](./media/adversarial-incorrect-clip.png)

Did you enable `cifar.preprocess()`? If sot, you can either add that in to normalise images into the interval [0, 1], or alternatively set the `clip_max=255` and similarly adjust `eps` to be about 1/10 of max image value: 25

You should be aiming for something looking more like this:

![](./media/adversarial-tensorboard-good.png)


## Utilising adversarial examples in training

### Practical 5.3: Adversarial training

We will now try to is to fold this knowledge back into the training process to produce a network that is robust against adversarial attacks. 

1. Compute adversarial examples for each batch of your **training images `trainImages`** 

2. Change:

    `$ for step in range(0, FLAGS.max_steps, 1):`
    
   To
   
    `$ for step in range(0, FLAGS.max_steps, 2):`
    
   Now run two training steps per iteration: one for your training examples and one for the adversarial training examples you computed in step 2.
   (tip: you can use the feed dict to populate tensors as well as placeholders). 
   
3. Evaluate the change in accuracy for both your original `testImages`, as well as your adversarial test images.

Interestingly, this adversarial training will result in...

1. **A slight drop of accuracy over your test set**
2. **An increase of accuracy over your adversarial test set - making the network more robust to adversarial attacks.**

## Preparing Lab 5 Portfolio

You should by now have the following files, which you can zip under the name `Lab_5_<username>.zip`

Note that we are asking you this lab to submit a copy of your *modified code with adversarial training*.

From your logs, include only the TensorBoard summaries and remove the checkpoints (`model.ckpt-*` files)

```
 Lab_5_<username>.zip
 |--cifar_adversarial.py
 |--adversarial_examples/
 |  |--x1.png
 |  |--x1_adv.png
 |  |--x1_diff.png
 |  |--x2.png
 |  |--x2_adv.png
 |  |--x2_diff.png
 |  |--x3.png
 |  |--x3_adv.png
 |  |--x3_diff.png
 |--logs/
 
```

Store this zip safely. You can now upload **all your labs' portfolios** to SAFE the specified deadline - check SAFE for exact deadline.

## Further resources

* [Explaining and Harnessing Adversarial Examples - Goodfellow et al.](https://arxiv.org/abs/1412.6572)
* [CleverHans](https://github.com/tensorflow/cleverhans)
* [CleverHans docs](http://cleverhans.readthedocs.io/en/latest/)
* [CleverHans CIFAR10 tutorial](https://github.com/tensorflow/cleverhans/blob/v2.0.0/examples/ex_cifar10_tf.py)
* [Intriguing properties of neural networks - Szegedy et al.](https://arxiv.org/abs/1312.6199)